In [ ]:
#Packages and hand-made math-y functions

import numpy as np
import torch
import random
import math
import time

def sgn(v):
  v1 = np.sign(v)
  v2 = np.where(v1 < 0, 0, v1)
  return v2

def choose(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

def MakeListElemsUnique(v):
  list_of_sets = [set(elem) for elem in v]
  unique_list_of_sets = []
  [unique_list_of_sets.append(x) for x in list_of_sets if x not in unique_list_of_sets]
  return unique_list_of_sets

def FindEdges(adj, order=False):
  edges = np.nonzero(adj)
  if len(adj.shape) == 2:
    edges = list(zip(edges[0],edges[1]))
  if len(adj.shape) == 3:
    edges = list(zip(edges[0],edges[1],edges[2]))
  if not order:
    return MakeListElemsUnique(edges)
  else:
    return edges

def GenerateG(degree,min_pos_edges,n):
  force_weights = True
  min_pos_edges = 4
  print_weights = True
  nonneg_edges = True

  if force_weights:
    G = np.zeros((n,n))
    for i in range(n):
      while np.sum(sgn(G[i])) < min_pos_edges:
        G[i] = np.zeros(n)
        num_nodes = random.randint(math.floor(.8*degree),math.floor(1.2*degree))
        while np.sum(np.absolute(G[i])) < num_nodes:
          G[i,random.randint(0,n-1)] = np.sign(random.random()*2 -1)
    G = np.sign(G)
    G = np.transpose(G)

  if not force_weights:
    G = np.zeros((n,n))
    for i in range(n):
      num_nodes = random.randint(math.floor(.8*degree),math.floor(1.2*degree))
      while np.sum(np.absolute(G[i])) < num_nodes:
        G[i,random.randint(0,n-1)] = np.sign(random.random()*2 -1)
    G = np.sign(G)
    G = np.transpose(G)

  if nonneg_edges:
    G = sgn(G)
  
  return G

def GenerateSamples(G, p_input, sample_num):
  samples = []
  for i in range(sample_num):
    h = np.zeros(n)
    num_ones = math.floor(n * p_input)
    while np.sum(h) < num_ones:
      h[random.randint(0,h.size-1)] = 1
    y = sgn(np.matmul(G,h))
    samples.append(y)
  return samples

def ThreewiseGraph(samples, p_input):
  N = len(samples)
  adj = np.zeros((n,n,n))
  a = 0
  for sample in samples:
    a += 1
    fired = np.argwhere(sample)   
    for i in fired:
      for j in fired:
        for k in fired:
          if i[0] != j[0] and i[0] != k[0] and j[0] != k[0]:
            adj[i[0],j[0],k[0]] += 1

  adj = np.where(adj <  2* p_input * N / 3, 0, adj)
  adj = np.where(adj > 0, 1, adj)
  return adj

def RecoverGraph3Wise(adj, degree):
  #Values
  edges = FindEdges(adj)
  edges_duplicate = FindEdges(adj)
  G1 = np.zeros((n,n))
  G1_index = 0
  counter = 0

  large_num = 10000 #to prevent endless loop
  while edges and counter < large_num:  
    counter += 1
    #Pick a random edge
    edge = edges[random.randint(0,len(edges)-1)]
    #print("edge: ",edge)
    edge_copy = edge.copy()
    v1 = edge_copy.pop()
    v2 = edge_copy.pop()
    v3 = edge_copy.pop()

    #Build S
    S = []
    for v in range(n):
      if {v,v1,v2} in edges_duplicate and {v,v2,v3} in edges_duplicate and {v,v1,v3} in edges_duplicate:          #edges_duplicate
        S.append(v)
    S.append(v1)
    S.append(v2)
    S.append(v3)

    #Build S1
    S1 = []
    if len(S) < 1.3 * degree  and len(S) > 0:

      #Find v's in S with enough neighbors for S1
      for v in S:
        neighbors = []
        for neighbor1 in S:
          for neighbor2 in S:
            if {v, neighbor1, neighbor2} in edges_duplicate:                                                     
              neighbors.append((neighbor1,neighbor2))
        
        neighbors = MakeListElemsUnique(neighbors)
        if len(neighbors) >= choose(math.floor(.8 * degree - 1),2):                                                
          S1.append(v)


      #Build G
      if len(S1) > 0:
        for v in S1:
          G1[v,G1_index] = 1
        G1_index += 1
        if G1_index > n-1:
          print("Index overload")
          return G1

      #Remove all edges in a community (S1) from edges_duplicate
      for v1 in S1:
        for v2 in S1:
          for v3 in S1:
            if {v1,v2,v3} in edges:             
              edges.remove({v1,v2,v3})

  return G1

def RecoveredColumns(G, G1):
  found_cols = 0
  G_t = np.transpose(G)
  G1_t = np.transpose(G1)
  for column in G_t:
    if (G1_t == column).all(1).any():
      found_cols += 1
  return found_cols/G.shape[0]

def StatError(G, G1, samples, sample_num, p_input):
  samples_rec = []
  for i in range(sample_num):
    h = np.zeros(n)
    num_ones = math.floor(n * p_input)
    while np.sum(h) < num_ones:
      h[random.randint(0,h.size-1)] = 1
    y = sgn(np.matmul(G1,h))
    samples_rec.append(y)
    
  cov = np.cov(samples,rowvar=False)
  cov_rec = np.cov(samples_rec,rowvar=False)
  diff = cov - cov_rec
  frob_norm_diff = np.linalg.norm(diff)
  frob_norm_cov = np.linalg.norm(cov)
  frob_norm_error = ( frob_norm_cov - frob_norm_diff ) / frob_norm_cov 

  mean = np.mean(np.mean(samples,axis=0))
  mean_diff = np.mean(np.mean(samples,axis=0)-np.mean(samples_rec,axis=0))
  mean_error = ( mean - mean_diff ) / (mean)

  return frob_norm_error, mean_error


In [ ]:

N = [25,50,100,150,200]
sample_NUM = [100,200,500,1000,5000]
p_inputs = [1/200,1/150,1/100,1/50,1/25]
degrees = [4, 5, 7, 9, 11]

threewise_time = []
recov_time = []
rec_cols = []
cov_errors = []
mean_errors = []

outer_matrix = []
for p_input in p_inputs:
  print(p_input)
  outer_vec = []
  for degree in degrees:
    print(degree)
    result_matrix = []
    for n in N:
      rec_cols = []
      cov_errors = []
      mean_errors = []

      result_vec = []
      for sample_num in sample_NUM:
        prob = degree/n 

        G = GenerateG(degree,4,n)
        samples = GenerateSamples(G, p_input, sample_num)

        threewise_start = time.time()
        adj = ThreewiseGraph(samples, p_input)
        threewise_stop = time.time()
        threewise_time.append(threewise_stop - threewise_start)

        recov_start = time.time()
        G1 = RecoverGraph3Wise(adj, degree)
        recov_stop = time.time()
        recov_time.append(recov_stop - recov_start)

        rec_cols.append(RecoveredColumns(G,G1))
        errors = StatError(G,G1,samples,sample_num,p_input)
        cov_errors.append(errors[0])
        mean_errors.append(errors[1])
        """
        print("RUN: ", n, sample_num)
        print(cov_errors[-1])
        print(mean_errors[-1])
        print(rec_cols[-1])
        """
        result_vec.append((str(int(100*rec_cols[-1]))+"%",mean_errors[-1],cov_errors[-1]))
      result_matrix.append(result_vec)
    outer_vec.append(result_matrix)
  outer_matrix.append(outer_vec)

0.005
4


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:180: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:184: RuntimeWarning: invalid value encountered in double_scalars


5
7
9
11


KeyboardInterrupt: ignored

In [ ]:
print(np.sum(G1))
#do for each degree and pinput, 4D matrix needed so maybe use dictionary?

In [ ]:
print(outer_matrix)
